For Google Colab users, setup the environment and install pypsa. 

In [ ]:
# !pip install pypsa pypsa[excel]

In [ ]:
# from google.colab import drive

# import os

# drive.mount('/content/drive')

# # Define your assignments folder path
# assignments_path = '/content/drive/MyDrive/mec4131z-assignments-2025'

# # Change working directory (optional, only if you want)
# os.chdir(assignments_path)


# Assignment 5 Notebook

Use this notebook for your assignment, there are plotting functions and the export code at the end. 

In [ ]:
import pypsa
import pandas as pd
pd.options.plotting.backend = "plotly"
n = pypsa.Network("assignment-5-case1.xlsx")
# n = pypsa.Network("assignment-5-case2.xlsx")
# n = pypsa.Network("assignment-5-case3.xlsx")
# n = pypsa.Network("assignment-5-case4.xlsx")
# n = pypsa.Network("assignment-5-case5.xlsx")
# n = pypsa.Network("assignment-5-case6.xlsx")

In [ ]:
n.generators_t.p_max_pu.plot()

In [ ]:
n.generators_t.p_min_pu.plot()

In [ ]:
n.loads_t.p_set.plot()

In [ ]:
n.optimize()

In [ ]:
n.generators_t.p["Loadshedding"].plot()

In [ ]:
n.generators_t.p["Loadshedding"].sum()


In [ ]:
n.generators_t.p.describe()

In [ ]:
n.storage_units_t.state_of_charge.plot()

In [ ]:
n.storage_units_t.p.plot()

In [ ]:
import pandas as pd

# Assume your network is called 'network'
# 1. Merge generator power time series with generator attributes
gen_p = n.generators_t.p.copy()
gen_p['carrier'] = n.generators.carrier

# 2. Sum across time
total_energy_per_generator = gen_p.drop(columns="carrier").sum()

# 3. Map generators to carriers
carrier_map = n.generators.carrier
total_energy_per_carrier = total_energy_per_generator.groupby(carrier_map).sum()

print(total_energy_per_carrier)


In [ ]:
import plotly.graph_objects as go

# 1. Map generators to their carrier
carrier_map = n.generators.carrier

# 2. Group actual production per snapshot
power_per_carrier_per_snapshot = n.generators_t.p.groupby(carrier_map, axis=1).sum()

# 3. Calculate installed capacity for specific carriers
carrier_capacities = n.generators.groupby('carrier').p_nom.sum()

# 4. Start building the figure
fig = go.Figure()

# 5. Add traces for each carrier's production
for carrier in power_per_carrier_per_snapshot.columns:
    fig.add_trace(go.Scatter(
        x=power_per_carrier_per_snapshot.index,
        y=power_per_carrier_per_snapshot[carrier],
        mode='lines',
        name=carrier
    ))

# 6. Add horizontal lines for coal, nuclear, oil capacities
for selected_carrier in ['coal', 'nuclear', 'oil']:
    if selected_carrier in carrier_capacities.index:
        capacity = carrier_capacities[selected_carrier]
        fig.add_trace(go.Scatter(
            x=[power_per_carrier_per_snapshot.index.min(), power_per_carrier_per_snapshot.index.max()],
            y=[capacity, capacity],
            mode='lines',
            name=f"{selected_carrier} capacity ({capacity:.0f} MW)",
            line=dict(dash='dash')  # dashed line
        ))

# 7. Layout settings
fig.update_layout(
    title="Power Production by Carrier with Installed Capacities",
    xaxis_title="Time",
    yaxis_title="Power [MW]",
    legend_title="Carriers and Capacities",
    template="plotly_white",
    height=600
)

fig.show()


In [ ]:
n.generators_t.p.plot(kind="area")

In [ ]:
n.storage_units_t.p.plot(kind="area")

In [ ]:
import pandas as pd

# Copy the storage p dataframe
storage_p = n.storage_units_t.p.copy()

# Create two new DataFrames for dispatch and charge
storage_dispatch = storage_p.clip(lower=0)  # Keep only positive (discharge)
storage_charge = (storage_p.clip(upper=0))  # Flip sign for charging

# Rename the indices to distinguish
storage_dispatch.index.name = 'snapshot'
storage_dispatch.columns = [f"{col} dispatch" for col in storage_dispatch.columns]

storage_charge.index.name = 'snapshot'
storage_charge.columns = [f"{col} charge" for col in storage_charge.columns]

# Now, bring in the generator p
generator_p = n.generators_t.p.copy()
generator_p.index.name = 'snapshot'

# Concatenate everything
combined_df = pd.concat([generator_p, storage_dispatch, storage_charge], axis=1)

# Optional: sort columns if you want it tidy
combined_df = combined_df.sort_index(axis=1)



In [ ]:
combined_df.plot(kind="area", title="Power Production and Storage Dispatch/Charge")

In [ ]:
combined_df.columns

In [ ]:
# Your desired plotting order
plot_order = [
    'BESS 1 charge',
    'BESS 2 charge',
    'Drakensberg charge',
    'Ingula charge',
    'Palmiet charge',
    'Steenbras charge',
    'Solar',
    'Wind',
    'Koeberg 2',
    'Koeberg 1',
    'Kusile 1',
    'Kusile 2',
    'Kusile 3',
    'Kusile 4',
    'Kusile 5',
    'Kusile 6',
    'Medupi 1',
    'Medupi 2',
    'Medupi 3',
    'Medupi 4',
    'Medupi 5',
    'Medupi 6',
    'Duvha 1',
    'Duvha 2',
    'Duvha 3',
    'Duvha 4',
    'Duvha 5',
    'Duvha 6',
    'Lethabo 1',
    'Lethabo 2',
    'Lethabo 3',
    'Lethabo 4',
    'Lethabo 5',
    'Lethabo 6',
    'Majuba 1',
    'Majuba 2',
    'Majuba 3',
    'Majuba 4',
    'Majuba 5',
    'Majuba 6',
    'Kendal 1',
    'Kendal 2',
    'Kendal 3',
    'Kendal 4',
    'Kendal 5',
    'Kendal 6',
    'Acacia',
    'Ankerlig',
    'Gas Engines',
    'Gourikwa',
    'Port Rex',
    'BESS 1 dispatch',
    'BESS 2 dispatch',
    'Drakensberg dispatch',
    'Ingula dispatch',
    'Palmiet dispatch',
    'Steenbras dispatch',
    'Loadshedding'
]

# Reorder the combined_df
# Drop missing ones first to avoid KeyError if any label doesn't exist
columns_in_df = [col for col in plot_order if col in combined_df.columns]

combined_df = combined_df[columns_in_df]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for col in combined_df.columns:
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df[col],
        mode='lines',
        stackgroup='one',
        name=col
    ))

fig.update_layout(
    title="Stacked Area Plot",
    xaxis_title="Time",
    yaxis_title="Power [MW]",
    legend_title="Units",
    hovermode="x unified"
)

fig.show()


In [ ]:
#to export results to excel

# n.export_to_excel("assignment-4-casex-results.xlsx")

In [ ]:
#to export results to ecsv

# n.export_to_csv("assignment-4-casex-results")